<a href="https://colab.research.google.com/github/JockWang/colab/blob/master/Wine_class%EF%BC%88%E5%89%AF%E6%9C%AC%EF%BC%89%EF%BC%88%E5%89%AF%E6%9C%AC%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 安装必要库文件

In [0]:
!pip install catboost

     |████████████████████████████████| 63.0MB 124kB/s 


## 加载必要库文件

In [0]:
import pandas as pd
import numpy as np
import catboost as cat
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import mean_absolute_error
from collections import Counter

## 读取训练、测试数据，并简单处理类别

In [0]:
train_data = pd.read_csv('/content/drive/My Drive/比赛/Wine_class/train.csv',sep=';')
test_data = pd.read_csv('/content/drive/My Drive/比赛/Wine_class/test.csv',sep=';')
cat_features = []
train_data['quality'] = train_data['quality']-3

In [0]:
train_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.3,0.19,0.27,13.9,0.057,45.0,155.0,0.99807,2.94,0.41,8.8,5
1,6.2,0.20,0.49,1.6,0.065,17.0,143.0,0.99370,3.22,0.52,9.2,3
2,7.1,0.23,0.35,16.5,0.040,60.0,171.0,0.99900,3.16,0.59,9.1,3
3,7.5,0.20,0.47,16.9,0.052,51.0,188.0,0.99944,3.09,0.62,9.3,2
4,7.0,0.15,0.38,15.3,0.045,54.0,120.0,0.99750,3.18,0.42,9.8,3


## Model
### Catboost十折交叉验证，单模96.5133

In [0]:
def CatKfoldResult(train=train_data.iloc[:,:-1],test=test_data,label=train_data.iloc[:,-1],cat_features=cat_features,K=10):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      eval_set = cat.Pool(data=valid_x,label=valid_y,cat_features=cat_features)
      model = cat.CatBoostClassifier(iterations=5000, learning_rate=0.01, depth=9, loss_function='MultiClass')
      model.fit(train_x,train_y,use_best_model=True,eval_set=eval_set,cat_features=cat_features,early_stopping_rounds=50)
      models.append(model)
      preds.append(model.predict(test).reshape((-1)))
  
  preds = np.array(preds)
  return models,preds.astype(int)

### XGBoost

首先，划分train、valid确定xgboost参数，之后使用十折交叉验证

In [0]:
# x_train,x_valid,y_train,y_valid = train_test_split(train_data.iloc[:,:-1],train_data.iloc[:,-1],test_size=0.1)
# xgb_train = xgb.DMatrix(x_train,y_train)
# xgb_valid = xgb.DMatrix(x_valid,y_valid)
# evallist = [(xgb_valid,'eval'),(xgb_train,'train')]

In [0]:
xgb_params = {
    'learning_rate': 0.3,
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'num_class': 7,
    'max_depth': 9,
    'subsample': .9,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'gamma': 0.,
    'alpha': 0.2,
    'lambda': 0.,
}

In [0]:
# model = xgb.train(xgb_params,dtrain=train,num_boost_round=2000,early_stopping_rounds=20,evals=evallist)

In [0]:
def XGBKfoldResult(train=train_data.iloc[:,:-1],test=test_data,label=train_data.iloc[:,-1],params=xgb_params,K=10):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  test = xgb.DMatrix(test.copy())
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      xgb_train = xgb.DMatrix(train_x,train_y)
      xgb_valid = xgb.DMatrix(valid_x,valid_y)
      evallist = [(xgb_valid,'eval')]
      model = xgb.train(params,dtrain=xgb_train,num_boost_round=2000,early_stopping_rounds=20,evals=evallist)
      models.append(model)
      preds.append(model.predict(test).reshape((-1)))
  preds = np.array(preds)
  return models,preds.astype(int)

### LightGBM

首先，划分训练集、测试集确定参数，之后十折交叉验证

In [0]:
# x_train,x_valid,y_train,y_valid = train_test_split(train_data.iloc[:,:-1],train_data.iloc[:,-1],test_size=0.1)
# train = lgb.Dataset(x_train,y_train)
# valid = lgb.Dataset(x_valid,y_valid)

In [0]:
lgb_params = {
    'objective': 'multiclass',
    'num_class': 7,
    'learning_rate': 0.01,
    'num_leaves': 127,
    'bagging_fraction': 1,
    'feature_fraction': .9,
    'lambda_l1': .1,
    'lambda_l2': 0,
}

In [0]:
# model = lgb.train(lgb_params,train_set=train,valid_sets=valid,num_boost_round=1000,early_stopping_rounds=20)

In [0]:
def LGBKfoldResult(train=train_data.iloc[:,:-1],test=test_data,label=train_data.iloc[:,-1],params=lgb_params,K=10):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      lgb_train = lgb.Dataset(train_x,train_y)
      lgb_valid = lgb.Dataset(valid_x,valid_y)
      model = lgb.train(params,train_set=lgb_train,valid_sets=lgb_valid,num_boost_round=1000,early_stopping_rounds=20)
      models.append(model)
      preds.append(model.predict(test).reshape((-1)))
  preds = np.array(preds)
  return models,preds.astype(int)

## 预测结果

预测采用两种策略：
* 三个Model交叉验证预测结果拼接到一起，选择预测众数
* 三个Model交叉验证预测结果选择预测众数，再从三个里选择众数



In [0]:
cat_models, cat_preds = CatKfoldResult()

In [0]:
xgb_models, xgb_preds = XGBKfoldResult()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


第 0 个Fold
[0]	eval-mlogloss:1.59491
Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	eval-mlogloss:1.41162
[2]	eval-mlogloss:1.3008
[3]	eval-mlogloss:1.21555
[4]	eval-mlogloss:1.13706
[5]	eval-mlogloss:1.08632
[6]	eval-mlogloss:1.04081
[7]	eval-mlogloss:1.00932
[8]	eval-mlogloss:0.996417
[9]	eval-mlogloss:0.976629
[10]	eval-mlogloss:0.967943
[11]	eval-mlogloss:0.951649
[12]	eval-mlogloss:0.941251
[13]	eval-mlogloss:0.928985
[14]	eval-mlogloss:0.930585
[15]	eval-mlogloss:0.925681
[16]	eval-mlogloss:0.918842
[17]	eval-mlogloss:0.918152
[18]	eval-mlogloss:0.914543
[19]	eval-mlogloss:0.915046
[20]	eval-mlogloss:0.917038
[21]	eval-mlogloss:0.919475
[22]	eval-mlogloss:0.92246
[23]	eval-mlogloss:0.927177
[24]	eval-mlogloss:0.931205
[25]	eval-mlogloss:0.929351
[26]	eval-mlogloss:0.929844
[27]	eval-mlogloss:0.933513
[28]	eval-mlogloss:0.931825
[29]	eval-mlogloss:0.935797
[30]	eval-mlogloss:0.941526
[31]	eval-mlogloss:0.946044
[32]	eval-mlogloss:0.948511
[33]	eval-mlogloss:0.9527

In [0]:
lgb_models, lgb_preds = LGBKfoldResult()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


第 0 个Fold
[1]	valid_0's multi_logloss: 1.28805
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's multi_logloss: 1.28395
[3]	valid_0's multi_logloss: 1.2808
[4]	valid_0's multi_logloss: 1.27796
[5]	valid_0's multi_logloss: 1.27423
[6]	valid_0's multi_logloss: 1.27043
[7]	valid_0's multi_logloss: 1.26726
[8]	valid_0's multi_logloss: 1.26431
[9]	valid_0's multi_logloss: 1.26077
[10]	valid_0's multi_logloss: 1.25718
[11]	valid_0's multi_logloss: 1.25389
[12]	valid_0's multi_logloss: 1.25094
[13]	valid_0's multi_logloss: 1.24769
[14]	valid_0's multi_logloss: 1.24455
[15]	valid_0's multi_logloss: 1.24129
[16]	valid_0's multi_logloss: 1.23807
[17]	valid_0's multi_logloss: 1.23496
[18]	valid_0's multi_logloss: 1.23247
[19]	valid_0's multi_logloss: 1.22972
[20]	valid_0's multi_logloss: 1.22642
[21]	valid_0's multi_logloss: 1.22339
[22]	valid_0's multi_logloss: 1.22084
[23]	valid_0's multi_logloss: 1.21789
[24]	valid_0's multi_logloss: 1.21474
[25]	valid_0's multi_loglo

In [0]:
def selectMost(preds):
  res = []
  for i in range(preds.shape[1]):
    res.append(Counter(preds[:,i]).most_common(1)[0][0])
  return np.array(res)
def saveRes(result):
  submit = []
  for i in range(1,result.shape[0]+1):
    submit.append([i, result[i-1]])
  pd.DataFrame(submit).to_csv('submit.csv',index=False,header=False)

### 策略一

In [0]:
preds = np.vstack((cat_preds,xgb_preds,lgb_preds))
result = selectMost(preds)
saveRes(result)

ValueError: ignored

### 策略二

In [0]:
res1, res2, res3 = selectMost(cat_preds), selectMost(xgb_preds), selectMost(lgb_preds)
preds = np.vstack((res1,res2,res3))
result = selectMost(preds)
saveRes(result)

ValueError: ignored

In [0]:
cat_preds.shape, lgb_preds.shape, xgb_preds.shape

((10, 1500), (10, 10500), (10, 1500))